<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.7

In [20]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  '''facilitates the Data Collection Process'''
  dl = DeepLearning()
  wr = Wrappers()
  if task == 'NLP':
    X2, y2, _ = wr.TCP(X, y)
    func = dl.RNN #insert CV after this
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(X, y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    df = dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  except:
    df = dl.CollectPerformance(params, func, X2, y2.reshape(-1), epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  size = []
  n_features = []
  batch = []
  depth = []
  for i in range(len(df)):
    size.append(X2.shape[0])
    n_features.append(X2.shape[1])
    if task == 'CV':
      depth.append(X2.shape[3])
  df['len_dataset'] = size
  df['n_features'] = n_features
  if task == 'CV':
    df['thickness'] = depth
  return df

In [23]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = load_breast_cancer()['target']
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
uploaded = files.upload()
data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
data.extractall()

Saving IMGDS1.zip to IMGDS1.zip


In [24]:
#image folder directories, if using image data
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']

In [38]:
Wikipedia

,Text,Human,AI
0,"aerospace is the human effort in science, engi...",1,0
1,a farmers market they are distinguished from p...,1,0
2,traditions by regiontraditions by schooltradit...,1,0
3,card or the card may refer to:,1,0
4,water skiing and a personal flotation device....,1,0
...,...,...,...
355,by jonathan rosen\n\njonathan rosen is a senio...,0,1
356,he was able to maintain good relations with t...,0,1
357,mealworms are shed during the larval phase of ...,0,1
358,"by\n\nby 2015, syrian president muammar gaddaf...",0,1


In [41]:
X = tweets
y = sentiments
df = Twitter
data_str = 'tweet'
output = len(np.unique(y))
embedding = dh.GetVocab(df, data_str)
rnn_grid = {'output_dim': [output], 'embedding': [embedding], 'nodes': [64], 'activation': ['tanh'], 'regularizer': [None, 'L1'], 
            'stacking': [False, True], 'dropout': [False], 'optimizer': ['adam'], 'method': ['GRU'], 'bidirectional': [False]}

X2 = articles
y2 = labels
df2 = Wikipedia
data_str2 = 'Text'
output2 = len(np.unique(y2))
embedding2 = dh.GetVocab(df2, data_str2)
rnn_grid2 = {'output_dim': [output2], 'embedding': [embedding2], 'nodes': [64], 'activation': ['tanh'], 'regularizer': [None], 
            'stacking': [False, True], 'dropout': [False, True], 'optimizer': ['adam'], 'method': ['GRU'], 'bidirectional': [False]}

#output = len(y)
#cnn_grid = {'output_dim': [output], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            #'pool':[2], 'dropout': [False, True], 'closer': [False], 'optimizer': ['adam', 'rmsprop'], 'metrics': ['accuracy']}

#output = len(np.unique(y))
#tc_grid = {'output_dim': [output], 'nodes': [64], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 
           #'dropout': [False], 'nlayers': [4, 5], 'closer': [False, True], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [32, 64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE', 'MAE'], 'optimizer': ['rmsprop'], 'y_col': ['string']}

df = BigDawg(X, y, rnn_grid, 'NLP')
df1 = BigDawg(X2, y2, rnn_grid2, 'NLP')
df2 = pd.concat([df, df1])
df2.shape

preforming 4 total fits ...
Epoch 1/50
preforming 4 total fits ...
Epoch 1/50
224/224 [==============================] - 5s 16ms/step - loss: 0.7472 - accuracy: 0.6564 - val_loss: 0.6394 - val_accuracy: 0.6667
Epoch 2/50
224/224 [==============================] - 3s 13ms/step - loss: 0.6384 - accuracy: 0.6686 - val_loss: 0.6599 - val_accuracy: 0.6572
Epoch 3/50
224/224 [==============================] - 3s 13ms/step - loss: 0.6035 - accuracy: 0.6941 - val_loss: 0.6614 - val_accuracy: 0.6449
Epoch 4/50
224/224 [==============================] - 3s 13ms/step - loss: 0.4814 - accuracy: 0.7767 - val_loss: 0.7492 - val_accuracy: 0.5984
Epoch 5/50
224/224 [==============================] - 3s 13ms/step - loss: 0.3777 - accuracy: 0.8228 - val_loss: 0.8344 - val_accuracy: 0.6051
Epoch 6/50
224/224 [==============================] - 3s 13ms/step - loss: 0.3220 - accuracy: 0.8417 - val_loss: 0.9766 - val_accuracy: 0.6012
1 of 4 fits complete!
Epoch 1/50
224/224 [==============================] -

(8, 18)

In [42]:
df2.to_csv('drive/MyDrive/Text_Classification_Results.csv')

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']